# Parking Simulator
In this simulation we simulate a parking garage nearby a supermarket.
- https://www.centrumparkeren.nl/hilversum/parkeergarages/gooiland
- https://www.google.com/search?q=albert+heijn+gooialdn&oq=albert+heijn+gooialdn&aqs=chrome..69i57j0l2.13159j0j4&sourceid=chrome&ie=UTF-8


In [ ]:
import simpy
import random
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

## CONSTANTinople
The birthplace of the constants

In [ ]:
RANDOM_SEED = 42
SIM_TIME = 24 * 60

# The day hours
DAY_MIN = 8 * 60
DAY_MAX = 18 * 60

# MINUTE rate
MINUTE_RATE = 0.2 / 5

# Max price day
DAY_PRICE = 8
# Max price night
NIGHT_PRICE = 5

# The peak hours
PEAK_MIN = 15 * 60   # 15:00
PEAK_MAX = 20 * 60   # 20:00

# The maximum capacity of the garage
MAX_CAPACITY = 364

# Constants for the amount of time a car is parked
MIN_CAR_PARK_TIME = 1
MAX_CAR_PARK_TIME = 60

# Constants for finding a spot
MIN_FINDING_SPOT = 1
MAX_FINDING_SPOT = 5

# Constants for exiting the garage
MIN_EXITING_TIME = 1
MAX_EXITING_TIME = 3

In [ ]:
class Garage(object):
    def __init__(self, env, max_spots=MAX_CAPACITY):
        self.env = env
        self.spots = simpy.Resource(env, max_spots)
        self.taken_spots = []
        self.timestamps = []
        
        # Process simulation events
        self.env.process(self.process())
        # Run the simulation
        self.env.run(until=SIM_TIME)
    
    def spot_queue(self):
        """Returns tuple with timestamp and amount of taken parking spots"""
        time = self.env.now
        queue = len(self.spots.users)
        self.taken_spots.append((time, queue))
        
        
    def car(self, name):
        """Details behaviour of a car"""
        time_of_arrival = self.env.now

        # Request one of the Garage's parking spots
        print('%s arriving at %d' % (name, self.env.now))
        with self.spots.request() as req:
            yield req

            time_of_assignment = self.env.now

            # Find spot
            print('%s finding parking spot %s' % (name, self.env.now))
            yield self.env.timeout(nrml_dist(MIN_FINDING_SPOT, MAX_FINDING_SPOT))

            # Park car
            print('%s starting to park %s' % (name, self.env.now))
            yield self.env.timeout(nrml_dist(MIN_CAR_PARK_TIME, MAX_CAR_PARK_TIME))

            # Leaving garage
            print('%s starting to leave %s' % (name, self.env.now))
            yield self.env.timeout(nrml_dist(MIN_EXITING_TIME, MAX_EXITING_TIME))

            # Left garage
            time_of_departure = self.env.now
            print('%s left %s' % (name, time_of_departure))

            # Append collected timestamps to self.timestamps
            self.timestamps.append((time_of_arrival, time_of_assignment, time_of_departure))
            
    def process(self):
        """"""
        time_now = 0
        car_numb = 0
        
        while True:
        
            yield self.env.timeout(1)

            if (PEAK_MIN <= time_now <= PEAK_MAX):
                total_new_cars = new_cars_amount(True)
            else:
                total_new_cars = new_cars_amount(False)
#             Why time_now and not self.env.now?
            time_now += 1

            for i in range(total_new_cars):
                self.env.process(self.car(car_numb))
                car_numb += 1
            self.spot_queue()

In [ ]:
def nrml_dist(min_, max_):
    """The function takes a minimal and maximal number and generates a number based on normal distribution"""
    mu = (max_ + min_) / 2
    sigma = mu / 3
    
    rd_numb = random.normalvariate(mu, sigma)
    
    if (rd_numb < min_):
        rd_numb = min_
    if (rd_numb > max_):
        rd_numb = max_
    
    return round(rd_numb)

In [ ]:
def new_cars_amount(is_peak):
    """Returns amount of cars based on is_peak"""
    
    total_cars = 0
    
    if is_peak:
        total_cars = 2
    else:
        total_cars = 1
        
    return total_cars

In [ ]:
def calc_price(ToA, ToD):
    """Calculates price based on: ToA: time of assignment, ToD: time of departure"""
    delta_t = abs(ToD - ToA)
    if DAY_MIN < ToA < DAY_MAX: 
    # Price inside the day interval
        price = delta_t * MINUTE_RATE
        if price > DAY_PRICE:
            price = DAY_PRICE
    else:
    # Price outside the day interval
        price = delta_t * MINUTE_RATE
        if price > NIGHT_PRICE:
            price = NIGHT_PRICE
    
    return round(price, 2)

In [ ]:
# For repeatability of the simulation we have chosen a random seed
random.seed(RANDOM_SEED)

# Create the environment
env = simpy.Environment()
garage = Garage(env)

data = garage.timestamps
taken_spots = garage.taken_spots

## Analysing the simulation

In [ ]:
received_money = list(map(lambda x : calc_price(x[1], x[2]), data))
print("total money received: {0}".format(round(sum(received_money), 2)))

In [ ]:
car_waiting_time = list(map(lambda x : abs(x[1] - x[0]), data))
# car_waiting_time

In [ ]:
car_parking_time = list(map(lambda x : abs(x[2] - x[1]), data))
# car_parking_time

In [ ]:
park_times = []

for i in range(1000):
    park_times.append(nrml_dist(MIN_CAR_PARK_TIME, MAX_CAR_PARK_TIME))

park_times.sort()
plt.hist(park_times)
plt.xlabel("Frequency")
plt.ylabel("Parking duration")
plt.show()

In [ ]:
taken_spots = np.array(taken_spots).T
plt.plot(taken_spots[0], taken_spots[1])
plt.xlabel("Time in minutes")
plt.ylabel("Used spots")
plt.ylim(0, MAX_CAPACITY)
plt.show()